In [1]:
import pandas as pd
import sys
import os


In [7]:

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
#from scripts.preprocess_cpv import process_cpv_data
from scripts.preprocess_categorielle import preprocess_categorielle, encoding_idAccordCadre
from scripts.preprocess_cpv import add_cpv_hierarchy_column
from scripts.preprocess_numerique import log_transformation, scale_numerical_features

ModuleNotFoundError: No module named 'scripts.preprocess_categorielle'

In [2]:
import pandas as pd
import numpy as np
import os
import sqlite3
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [3]:
df = pd.read_csv('../data/train.csv', encoding='utf-8')

In [4]:
df

,uid,id,nature,acheteur_id,acheteur_nom,acheteur_siren,titulaire_id,titulaire_typeIdentifiant,titulaire_nom,titulaire_siren,...,ccag,sousTraitanceDeclaree,typeGroupementOperateurs,tauxAvance,origineUE,origineFrance,lieuExecution_code,lieuExecution_typeCode,idAccordCadre,source_open_data
0,215706722000182024TH240,2024TH240,Marché,21570672200018,COMMUNE DE THIONVILLE,215706722.0,42153568300013,SIRET,TRAVAUX SOUTERRAINS ELECTRIQUES ET GAZIERS,421535683.0,...,Travaux,0.0,Pas de groupement,0.05,0.0,0.0,57100,Code postal,NaN,data.gouv.fr decp-2025-04.json
1,200052090000122023ELECF8,2023ELECF8,Marché,20005209000012,SYNDICAT INTERCOMMUNALD'ENERGIES DU DEPARTEMEN...,200052090.0,53791596900134,SIRET,CEGELEC RODEZ,537915969.0,...,Pas de CCAG,NaN,Pas de groupement,0.00,0.0,0.0,12032,Code postal,NaN,data.gouv.fr decp-2025-04.json
2,200058519001132023-0202,2023-0202,Marché,20005851900113,CA SAINT GERMAIN BOUCLES DE SEINE,200058519.0,42094822600048,SIRET,EUROVIA ILE DE FRANCE,420948226.0,...,Travaux,0.0,Conjoint,NaN,NaN,NaN,78230,Code postal,2023-0202,data.gouv.fr decp-2024.json
3,239740012000122023602099,2023602099,Marché,23974001200012,REGION REUNION,239740012.0,50067844600047,SIRET,TOPO SERVICES,500678446.0,...,Pas de CCAG,NaN,Pas de groupement,NaN,NaN,NaN,04,Code région,2023602099,data.gouv.fr decp-2024.json
4,2631001250001620241941,20241941,Marché,26310012500016,CENTRE HOSPITALIER UNIVERSITAIRE DE TOULOUSE,263100125.0,77566225700150,SIRET,SANOFI WINTHROP INDUSTRIE,775662257.0,...,Fournitures courantes et services,NaN,Pas de groupement,0.00,0.0,0.0,76,Code région,NaN,data.gouv.fr decp-2025-03.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239731,256701152000252024S0055MPAOC,2024S0055MPAOC,Marché,25670115200025,SYNDICAT MIXTE DES EAUX ET DE L'ASSAINISSEMENT...,256701152.0,50217952600038,SIRET,T&S TECHNOLOGY &STRATEGY I.T.,502179526.0,...,Pas de CCAG,0.0,Pas de groupement,NaN,NaN,NaN,44,Code région,2023S0156MAOOC,data.gouv.fr decp-2024.json
239732,216901231000112023O4484,2023O4484,Marché,21690123100011,COMMUNE DE LYON,216901231.0,30389008100126,SIRET,CARDEM,303890081.0,...,Pas de CCAG,1.0,Pas de groupement,NaN,NaN,NaN,69123,Code commune,NaN,data.gouv.fr decp-2024.json
239733,211202965000112024001L06,2024001L06,Marché,21120296500011,COMMUNE DE VIALA DU TARN,211202965.0,53920081600012,SIRET,MENUISERIE DURAND,539200816.0,...,Pas de CCAG,NaN,Pas de groupement,NaN,NaN,NaN,12490,Code postal,NaN,data.gouv.fr decp-2024.json
239734,2113010070001220240701,20240701,Marché,21130100700012,COMMUNE DE SAINT REMY DE PROVENCE,211301007.0,32344748200015,SIRET,SELE,323447482.0,...,Travaux,0.0,Conjoint,0.05,0.0,0.0,13210,Code postal,NaN,data.gouv.fr decp-2025-04.json


In [5]:
df.isnull().sum()

uid                               0
id                                0
nature                            0
acheteur_id                       0
acheteur_nom                    154
acheteur_siren                  137
titulaire_id                      0
titulaire_typeIdentifiant         0
titulaire_nom                  6037
titulaire_siren                 912
objet                             0
montant                           0
codeCPV                           0
procedure                         0
dureeMois                         6
dateNotification                  0
datePublicationDonnees            2
formePrix                         0
attributionAvance            139179
offresRecues                  81034
marcheInnovant               141146
ccag                              0
sousTraitanceDeclaree        115718
typeGroupementOperateurs          0
tauxAvance                   103643
origineUE                    104388
origineFrance                104388
lieuExecution_code          

In [8]:
from scripts.preprocess_final import create_preprocessing_pipeline_init, create_preprocessing_pipeline_follow, create_complete_pipeline

In [10]:
pipeline_prepa = create_preprocessing_pipeline_init(cat='marche_sim')


In [11]:
pipeline_prepa

Pipeline(steps=[('id_accord_encoder', IdAccordCadreEncoder()),
                ('taux_avance_categorizer', TauxAvanceCategorizer()),
                ('outliers_feature_rows_selector',
                 InitTransformer(cat='marche_sim'))])

In [12]:

# Fit and transform your data
X_transformed = pipeline_prepa.fit_transform(df)

In [13]:
X_transformed

,procedure,dureeMois,nature,formePrix,offresRecues,ccag,sousTraitanceDeclaree,typeGroupementOperateurs,tauxAvance_cat,origineFrance,idAccordCadre,montant,marcheInnovant,codeCPV_2
0,Appel d'offres ouvert,23.0,Marché,Unitaire,2.0,Travaux,0.0,Pas de groupement,small_advance,0.0,0,106866.59,0.0,45000000
1,Appel d'offres ouvert,36.0,Marché,Mixte,NaN,Pas de CCAG,NaN,Pas de groupement,no_advance,0.0,0,6000000.00,NaN,45000000
2,Appel d'offres ouvert,12.0,Marché,Forfaitaire,2.0,Travaux,0.0,Conjoint,NaN,NaN,1,2200000.00,0.0,45000000
3,Appel d'offres ouvert,48.0,Marché,Mixte,NaN,Pas de CCAG,NaN,Pas de groupement,NaN,NaN,1,2400000.00,NaN,71000000
4,Procédure adaptée,36.0,Marché,Unitaire,NaN,Fournitures courantes et services,NaN,Pas de groupement,no_advance,0.0,0,2268857.60,NaN,33000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239731,Procédure adaptée,12.0,Marché,Unitaire,NaN,Pas de CCAG,0.0,Pas de groupement,NaN,NaN,1,50400.00,0.0,72000000
239732,Appel d'offres ouvert,9.0,Marché,Mixte,NaN,Pas de CCAG,1.0,Pas de groupement,NaN,NaN,0,440742.36,NaN,45000000
239733,Procédure adaptée,16.0,Marché,Unitaire,NaN,Pas de CCAG,NaN,Pas de groupement,NaN,NaN,0,77221.40,NaN,45000000
239734,Procédure adaptée,24.0,Marché,Forfaitaire,5.0,Travaux,0.0,Conjoint,small_advance,0.0,0,212125.00,0.0,45000000


In [14]:
X_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 227669 entries, 0 to 239735
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   procedure                 227669 non-null  object  
 1   dureeMois                 227663 non-null  float64 
 2   nature                    227669 non-null  object  
 3   formePrix                 227669 non-null  object  
 4   offresRecues              148883 non-null  float64 
 5   ccag                      227669 non-null  object  
 6   sousTraitanceDeclaree     114595 non-null  float64 
 7   typeGroupementOperateurs  227669 non-null  object  
 8   tauxAvance_cat            127068 non-null  category
 9   origineFrance             126306 non-null  float64 
 10  idAccordCadre             227669 non-null  int64   
 11  montant                   227669 non-null  float64 
 12  marcheInnovant            89514 non-null   float64 
 13  codeCPV_2                 227669 n

In [15]:
pipeline_final = create_preprocessing_pipeline_follow()

In [16]:
pipeline_final

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('duree_mois_dropper', DureeMoisDropper()),
                                 ('column_transformer',
                                  ColumnTransformer(transformers=[('offres_recues_pipeline',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='median')),
                                                                                   ('log_transform',
                                                                                    LogTransformer()),
                                                                                   ('scaler',
                                                                                    StandardScaler())]),
                                                                   ['offresRecues']),
                                                                  ('other_num_pipeline',
                                                                   Pipeline(step...
                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                  sparse_output=False))]),
                                                                   ['nature',
                                                                    'procedure',
                                                                    'formePrix',
                                                                    'marcheInnovant',
                                                                    'ccag',
                                                                    'sousTraitanceDeclaree',
                                                                    'typeGroupementOperateurs',
                                                                    'origineFrance',
                                                                    'idAccordCadre',
                                                                    'codeCPV_2',
                                                                    'tauxAvance_cat'])]))])),
                ('to_dataframe',
                 FunctionTransformer(func=<function create_preprocessing_pipeline_follow.<locals>.<lambda> at 0x3442fa4d0>))])

In [17]:
X2_transformed = pipeline_final.fit_transform(X_transformed)
X2_transformed

,offres_recues_pipeline__offresRecues,other_num_pipeline__montant,other_num_pipeline__dureeMois,cat_pipeline__nature_Marché,cat_pipeline__nature_Marché de défense ou de sécurité,cat_pipeline__nature_Marché de partenariat,cat_pipeline__procedure_Appel d'offres ouvert,cat_pipeline__procedure_Appel d'offres restreint,cat_pipeline__procedure_Dialogue compétitif,cat_pipeline__procedure_Marché passé sans publicité ni mise en concurrence préalable,...,cat_pipeline__codeCPV_2_80000000,cat_pipeline__codeCPV_2_85000000,cat_pipeline__codeCPV_2_90000000,cat_pipeline__codeCPV_2_92000000,cat_pipeline__codeCPV_2_98000000,cat_pipeline__tauxAvance_cat_large_advance,cat_pipeline__tauxAvance_cat_medium_advance,cat_pipeline__tauxAvance_cat_missing,cat_pipeline__tauxAvance_cat_no_advance,cat_pipeline__tauxAvance_cat_small_advance
0,-0.578746,-0.511996,0.153094,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.032831,2.356890,0.616194,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.578746,1.642289,-0.502836,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-0.032831,1.704263,0.916718,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.032831,1.664240,0.616194,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227658,-0.032831,-1.047308,-0.502836,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
227659,-0.032831,0.497170,-0.783526,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
227660,-0.032831,-0.743407,-0.215833,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
227661,0.736593,-0.023685,0.196768,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
X2_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227663 entries, 0 to 227662
Columns: 173 entries, offres_recues_pipeline__offresRecues to cat_pipeline__tauxAvance_cat_small_advance
dtypes: float64(173)
memory usage: 300.5 MB


In [9]:
pipeline_total = create_complete_pipeline(cat='marche_sim', min=20000, max=50000000, top_n=40, level=2)


In [10]:
pipeline_total

FunctionTransformer(func=<function create_complete_pipeline.<locals>.process_with_both_pipelines at 0x331cdec20>)

In [11]:
X3_transformed = pipeline_total.fit_transform(df)
X3_transformed

/Users/loicdogon/.pyenv/versions/3.10.6/envs/decp_ml_env/lib/python3.10/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


NotFittedError: Pipeline is not fitted yet.

In [12]:
pipeline_ultime = create_complete_pipeline(
    cat=['marche_sim'],
    min=20000,
    max=50000000,
    top_n=40,
    level=2
)


X_processed = pipeline_ultime.fit_transform(df)
print(f"After preprocessing: {X_processed.shape[0]} rows and {X_processed.shape[1]} columns")
X_processed.head()

/Users/loicdogon/.pyenv/versions/3.10.6/envs/decp_ml_env/lib/python3.10/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


AttributeError: 'str' object has no attribute 'dropna'

# ultime test

In [1]:
import pandas as pd
import numpy as np
import os

# Import your preprocessing module
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from scripts.preprocess_final import create_preprocessing_pipeline_init, create_preprocessing_pipeline_follow, create_complete_pipeline

In [4]:
df = pd.read_csv('../data/train.csv', encoding='utf-8')

In [7]:
pipeline = create_complete_pipeline(
    cat='marche_sim',
    min=20000,
    max=50000000,
    top_n=40,
    level=2
)


In [8]:

X_processed = pipeline.fit_transform(df)


In [57]:

print(f"After preprocessing: {X_processed.shape[0]} rows and {X_processed.shape[1]} columns")
X_processed

After preprocessing: 148875 rows and 74 columns


,offres_recues_pipeline__offresRecues,other_num_pipeline__montant,other_num_pipeline__dureeMois,other_num_pipeline__origineFrance,binary_pipeline__marcheInnovant,binary_pipeline__sousTraitanceDeclaree,binary_pipeline__idAccordCadre,cat_pipeline__nature_Marché,cat_pipeline__nature_Marché de défense ou de sécurité,cat_pipeline__nature_Marché de partenariat,...,cat_pipeline__codeCPV_2_80000000,cat_pipeline__codeCPV_2_85000000,cat_pipeline__codeCPV_2_90000000,cat_pipeline__codeCPV_2_92000000,cat_pipeline__codeCPV_2_98000000,cat_pipeline__tauxAvance_cat_large_advance,cat_pipeline__tauxAvance_cat_medium_advance,cat_pipeline__tauxAvance_cat_missing,cat_pipeline__tauxAvance_cat_no_advance,cat_pipeline__tauxAvance_cat_small_advance
0,-0.482391,-0.488767,0.176086,-0.108812,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.482391,1.662777,-0.489474,-0.108812,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-1.105195,0.738542,0.950921,-0.108812,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.582300,1.061679,-1.016521,-0.108812,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.302250,-0.247586,-0.489474,-0.108812,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148870,1.024187,0.849022,0.950921,-0.108812,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
148871,-1.105195,-0.703353,0.220401,-0.108812,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
148872,0.302250,-0.277159,-2.081268,-0.108812,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
148873,1.205104,0.760630,-0.489474,-0.108812,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
#exporter en csv X_processed
X_processed.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'data', 'train_preprocessed.csv'), index=False)